In [7]:
# notebook: notebooks/modelo_previsao_magnitude.ipynb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import os

# 1. Carregar os dados
df = pd.read_csv('../dados_processados/terremotos_processados.csv', parse_dates=['data_hora'])

# 2. Pré-processamento
# Remover registros com valores nulos em colunas importantes
df.dropna(subset=['latitude', 'longitude', 'profundidade_km', 'magnitude', 'local'], inplace=True)

# 3. Criar features a partir da data
df['ano'] = df['data_hora'].dt.year
df['mes'] = df['data_hora'].dt.month
df['dia_semana'] = df['data_hora'].dt.dayofweek
df['hora'] = df['data_hora'].dt.hour

# 4. Codificar variável categórica 'local'
le_local = LabelEncoder()
df['local_cod'] = le_local.fit_transform(df['local'])

# 5. Definir features (X) e target (y)
X = df[['latitude', 'longitude', 'profundidade_km', 'local_cod', 'ano', 'mes', 'dia_semana', 'hora']]
y = df['magnitude']

# 6. Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. Treinar modelo
modelo = RandomForestRegressor(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# 8. Avaliação do modelo
y_pred = modelo.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("Avaliação do Modelo:")
print(f"MAE: {mae:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")

# 9. Salvar o modelo e os dados preditivos
os.makedirs('../modelos', exist_ok=True)
joblib.dump(modelo, '../modelos/modelo_magnitude.pkl')
df.to_parquet('../dados_processados/terremotos_features.parquet', index=False)

print("\nModelo salvo em '../modelos/modelo_magnitude.pkl'")
print("Dados salvos em '../dados_processados/terremotos_features.parquet'")


Avaliação do Modelo:
MAE: 0.292
RMSE: 0.405
R²: 0.025

Modelo salvo em '../modelos/modelo_magnitude.pkl'
Dados salvos em '../dados_processados/terremotos_features.parquet'
